In [1]:
import pandas as pd

In [73]:
db_credit = pd.read_excel(r'C:\Users\Felipe\Desktop\Credit Card\credit.xls')

In [74]:
db_credit.drop(columns=['X12','X13','X14','X15', 'X16', 'X17', 'X18', 'X19','X20','X21','X22', 'X23', 'Y'], inplace = True)

In [75]:
db_credit.head()

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11
0,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6
1,1,20000,2,2,1,24,2,2,-1,-1,-2,-2
2,2,120000,2,2,2,26,-1,2,0,0,0,2
3,3,90000,2,2,2,34,0,0,0,0,0,0
4,4,50000,2,2,1,37,0,0,0,0,0,0


In [76]:
db_credit.columns

Index(['Unnamed: 0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9',
       'X10', 'X11'],
      dtype='object')

In [77]:
db_credit.rename(columns={'Unnamed: 0':'id', 'X1':'limit_bal', 'X2':'sex', 'X3':'education', 'X4':'marriage',
                          'X5': 'age', 'X6':'september', 'X7':'august', 'X8':'july', 'X9':'june',
                          'X10':'march', 'X11':'april'}, inplace = True)
db_credit.drop(0, axis = 0, inplace = True)
db_credit.head()

,id,limit_bal,sex,education,marriage,age,september,august,july,june,march,april
1,1,20000,2,2,1,24,2,2,-1,-1,-2,-2
2,2,120000,2,2,2,26,-1,2,0,0,0,2
3,3,90000,2,2,2,34,0,0,0,0,0,0
4,4,50000,2,2,1,37,0,0,0,0,0,0
5,5,50000,1,2,1,57,-1,0,-1,0,0,0
